# 참고 코드

1. 우선 순위 순서대로

https://github.com/skykim/dialogpt-finetuning-ko

https://dagshub.com/kingabzpro/DailoGPT-RickBot/src/main

https://huggingface.co/byeongal/Ko-DialoGPT

https://colab.research.google.com/github/ncoop57/i-am-a-nerd/blob/master/_notebooks/2020-05-12-chatbot-part-1.ipynb#scrollTo=uieRDgEmhoyH

https://colab.research.google.com/drive/1cAtfkbhqsRsT59y3imjR1APw3MHDMkuV?usp=sharing#scrollTo=BAd_D_o6Py1b


# 레퍼런스
https://github.com/microsoft/DialoGPT  
https://paperswithcode.com/task/open-domain-dialog#datasets  
https://www.microsoft.com/en-us/research/project/large-scale-pretraining-for-response-generation/   
https://github.com/huggingface/transformers/tree/main/examples/pytorch

In [63]:
!pip install sacrebleu[ko]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 118 kB 6.3 MB/s            
     |████████████████████████████████| 582 kB 40.2 MB/s            
     |████████████████████████████████| 33.2 MB 59.9 MB/s            ��██████████████▊            | 20.4 MB 59.9 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for mecab-ko-dic: filename=mecab_ko_dic-1.0.0-py3-none-any.whl size=33424414 sha256=92df5ff50b91d59b48dfd7811a8a6a28a1c92429634c2b99a11de3dff00b7ed0
  Stored in directory: /aiffel/.cache/pip/wheels/1e/26/c0/ed4c061096b1480abefe1e2a0356543bf7a38f61c037b69ab6
Successfully built mecab-ko-dic


In [1]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple
import json
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

import torch
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
import wandb

import datasets
from datasets import load_metric

from transformers import (
    WEIGHTS_NAME,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

In [2]:
# Arguments
class Args():
    def __init__(self):
        self.output_dir = 'new_data_4'
        self.model_type = 'gpt2'
        
        self.model_name_or_path =  'byeongal/Ko-DialoGPT'
        self.config_name =  'byeongal/Ko-DialoGPT'
        self.tokenizer_name = 'byeongal/Ko-DialoGPT'
        
        
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 8
        self.per_gpu_eval_batch_size = 8
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 30
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 233060
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

# Configs
logger = logging.getLogger(__name__)


In [5]:
import os
# GPU 사용 가능 -> True, GPU 사용 불가 -> False
print(torch.cuda.is_available())

# GPU 사용 가능 -> 가장 빠른 번호 GPU, GPU 사용 불가 -> CPU 자동 지정 예시
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

# GPU 이름 체크(cuda:0에 연결된 그래픽 카드 기준)
print(torch.cuda.get_device_name(0)) # 'NVIDIA TITAN X (Pascal)'

# # # 사용 가능 GPU 개수 체크
print(torch.cuda.device_count()) # 3


# 2번 GPU만 사용하고 싶은 경우 예시(cuda:0에 지정)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

True
Tesla T4
1


In [18]:
def metric_bleu():
    tokenizer = AutoTokenizer.from_pretrained('new_data_4')
    model = AutoModelForCausalLM.from_pretrained('new_data_4')

    test_file = pd.read_csv('qa_faq_test_ver1.csv')  # 경로 설정
    bleu_metric = load_metric("sacrebleu")
    scores = []
    for i in tqdm(range(len(test_file))):
        step = 0
        q = test_file['Q'][i]
        a = test_file['A'][i]

        new_user_input_ids = tokenizer.encode(
            q + tokenizer.eos_token, return_tensors='pt')
        bot_input_ids = torch.cat(
            [chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

        chat_history_ids = model.generate(
            bot_input_ids, max_length=1000,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
            do_sample=True,
            top_k=100,
            top_p=0.7,
            temperature=0.8
        )
        # 예측한 문장
        pred = tokenizer.decode(
            chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        references = []
        references.append(a)             # 참조 문장

        bleu_metric.add(prediction=pred, reference=references)
        results = bleu_metric.compute(smooth_method='exp', tokenize='ko-mecab')
        results['precisions'] = [np.round(p, 2) for p in results['precisions']]
        scores.append(results['precisions'])

    return (np.mean(scores))

100%|██████████| 404/404 [06:23<00:00,  1.05it/s]

32.41388613861386


In [7]:
def preprocess():
    
    train = pd.read_csv('qa_faq_train_ver1.csv')
    validate = pd.read_csv('qa_faq_test_ver1.csv')

    train_contexted = []
    validate_contexted = []
    for i in range(len(train)):
        row = []
        row.append(train.loc[i][1])
        row.append(train.loc[i][0])
        train_contexted.append(row)  

    for i in range(len(validate)):
        row = []
        row.append(validate.loc[i][1])
        row.append(validate.loc[i][0])
        validate_contexted.append(row)

    columns = ['response', 'context'] 
    columns = columns + ['context/'+str(i) for i in range(0)]
      #print(columns)

    len(train_contexted)
    trn_df = pd.DataFrame.from_records(train_contexted, columns=columns)
      #print(trn_df.head(5))

    len(validate_contexted)
    val_df = pd.DataFrame.from_records(validate_contexted, columns=columns)
      #print(val_df.head(5))

    return trn_df, val_df

## Dataset Loader
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

# Cacheing and storing of data/checkpoints
def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))
    
    for path in glob_checkpoints:
        
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9])".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted

def rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

## Training and Evaluating
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    #if args.n_gpu > 1:
    #    model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for itr in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

#             if args.n_gpu > 1:
#                 loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item() # <tensor (1),regu- >
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss
                    

                if args.local_rank in [-1, 0] and args.save_steps > 0 and args.num_train_epochs == (itr+1) and (step+1) == len(train_dataloader):
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}-{}".format(checkpoint_prefix, global_step, itr+1))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
#         wandb.log({
#         "global_step": global_step,
#         "logging_loss": logging_loss, "loss": (tr_loss - logging_loss) / args.logging_steps,
#         "average_loss": tr_loss / global_step,"lr": scheduler.get_last_lr()[0]})

        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()
        

    return global_step, tr_loss / global_step

# Evaluation of some model
def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
#     if args.n_gpu > 1:
#         model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))
    bleu = metric_bleu()
    
#     wandb.log({
#         "eval_loss": eval_loss,
#         "perplexity": perplexity})#,"BLEU score": bleu})
    
    result = {"perplexity": perplexity, "BLEU score": bleu}

    output_eval_file = os.path.join(
        eval_output_dir, prefix, "eval_results_new_all.txt")
    
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

def main():
    # wandb.init()
    if args.should_continue:
        checkpoints_sorted = sorted_checkpoints(args)
        print(checkpoints_sorted)
        if len(checkpoints_sorted) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = checkpoints_sorted[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Prepare data
    df_trn, df_val = preprocess()

    # Setup CUDA, GPU & distributed training
    args.n_gpu = torch.cuda.device_count()
    args.device = device
    
    
    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)
#     wandb.config.update(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    #wandb.watch(model)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)
#         wandb.log({
#         "global_step": global_step,
#         "logging_loss": logging_loss,
#         "average loss": tr_loss / global_step})

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)
   
    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

if __name__ == '__main__':
    main()

02/06/2023 05:25:51 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
02/06/2023 05:26:01 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7ff6204809d0>
02/06/2023 05:26:01 - INFO - __main__ -   Creating features from dataset file at cached
02/06/2023 05:26:01 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
02/06/2023 05:26:01 - INFO - __main__ -   ***** Running training *****
02/06/2023 05:26:01 - INFO - __main__ -     Num examples = 1611
02/06/2023 05:26:01 - INFO - __main__ -     Num Epochs = 30
02/06/2023 05:26:01 - INFO - __main__ -     Instantaneous batch size per GPU = 8
02/06/2023 05:26:01 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 8
02/06/2023 05:26:01 - INFO - __main__ -     Gradient Accumulation steps = 1
02/06/2023 05:26:01 - INFO - __main__ -     Total optimization steps = 6030
Iteration:  19

Iteration:  46%|████▌     | 92/201 [00:20<00:22,  4.76it/s]


Iteration:  73%|███████▎  | 146/201 [00:33<00:10,  5.24it/s]


Iteration:  97%|█████████▋| 195/201 [00:43<00:01,  4.78it/s]02/06/2023 05:29:52 - INFO - __main__ -   Saving model checkpoint to new_data_4/checkpoint-1000-5
02/06/2023 05:29:54 - INFO - __main__ -   Saving optimizer and scheduler states to new_data_4/checkpoint-1000-5

Iteration:  98%|█████████▊| 197/201 [00:47<00:03,  1.21it/s]


Iteration:  25%|██▍       | 50/201 [00:12<00:42,  3.53it/s]


...

Iteration:  38%|███▊      | 77/201 [00:18<00:34,  3.56it/s]


Iteration:  65%|██████▌   | 131/201 [00:29<00:20,  3.39it/s]


Iteration:  85%|████████▍ | 170/201 [00:38<00:07,  4.39it/s]02/06/2023 05:49:04 - INFO - __main__ -   Saving model checkpoint to new_data_4/checkpoint-6000-30
02/06/2023 05:49:06 - INFO - __main__ -   Saving optimizer and scheduler states to new_data_4/checkpoint-6000-30

Epoch: 100%|██████████| 30/30 [23:11<00:00, 46.38s/it]
02/06/2023 05:49:13 - INFO - __main__ -    global_step = 6030, average loss = 0.17837355273944105
02/06/2023 05:49:13 - INFO - __main__ -   Saving model checkpoint to new_data_4
02/06/2023 05:49:28 - INFO - __main__ -   Evaluate the following checkpoints: ['new_data_4']
02/06/2023 05:49:29 - INFO - __main__ -   Creating features from dataset file at cached
02/06/2023 05:49:29 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
02/06/2023 05:49:29 - INFO - __main__ -   ***** Running evaluation  *****
02/06/2023 05:49:29 - INFO - __main__ -     Num examples = 404
02/06/2023 05:49:29 - INFO - __main__ -     Batch size = 8
Evaluating: 100%

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

100%|██████████| 404/404 [06:33<00:00,  1.03it/s]
02/06/2023 05:56:10 - INFO - __main__ -   ***** Eval results  *****
02/06/2023 05:56:10 - INFO - __main__ -     BLEU score = 33.04744430693069
02/06/2023 05:56:10 - INFO - __main__ -     perplexity = tensor(2.0406)


In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.utils import logging
import torch

#ignore warning messages
logging.set_verbosity_error()

tokenizer = AutoTokenizer.from_pretrained('new_data_4')
model = AutoModelForCausalLM.from_pretrained('new_data_4')
step = 0
while True:
    if step > 4:
        break
    new_user_input_ids = tokenizer.encode(input(">> User:")+tokenizer.eos_token, return_tensors='pt')
    #print(new_user_input_ids)
    # append the new user input tokens to the chat histocry
    bot_input_ids = new_user_input_ids#torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1024,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=100,       
        do_sample=True, 
        top_k=200, 
        top_p=0.3,
        temperature=0.8
    )

    # pretty print last ouput tokens from bot
    print("socarBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    step+=1

>> User:사고 났어요
socarBot: 괜찮으신가요 사진 전송해주세요. 사고 발생 시 쏘카 사고센터로 사고 접수를 진행해주세요.
>> User:사고
socarBot: 사고가 나셨군요. 사고가 난 차량 부위의 사진첨부해주세요
>> User:환불
socarBot: 쏘카 앱에서 반납을 연장할 수 있습니다.
>> User:반납
socarBot: 쏘카 앱 스마트키에서 반납하기를 누르고 어디에 어떻게 주차했는지 사진과 설명을 남기면 반납이 완료됩니다.
>> User:쏘카
socarBot: 쏘카존 위치는 예약 시 또는 예약 후 쏘카 앱을 통해서 확인할 수 있습니다.
